In [0]:
#importing the packages
import pandas as pd
import numpy as np
import re
import gensim
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordDetokenizer
from textblob import TextBlob
from mlxtend.frequent_patterns import apriori,association_rules
from nltk.tokenize import TweetTokenizer
import spacy
import collections
from operator import itemgetter
from collections import  OrderedDict

In [0]:
#reading the data
review_data = pd.read_csv('reviews-all_reviews-default-2019-06-23.csv')

In [0]:
review_data.head()

,ASIN,Country Code,Product Title,Product Review URL,Review Rating,Review Date,Reviewer URL,Reviewer Name,Reviewer ID,Review Title,Review Text,Verified Review,Variation ASIN,Deleted,Colour,Size
0,B07DJHV6VZ,IN,"OnePlus 6T (Mirror Black, 8GB RAM, 128GB Storage)",https://www.amazon.in/gp/review/R10PXUVIR6L0MO,4.0,2019-06-22,https://www.amazon.in/gp/profile/amzn1.account...,nakul,AGUQQEZD6QF55SUG4OIWZLAZ2JKQ,Nice phone,Good,Yes,B07DJHY82F,Not Deleted,Midnight Black,8GB/128 GB
1,B07DJHV6VZ,IN,"OnePlus 6T (Mirror Black, 8GB RAM, 128GB Storage)",https://www.amazon.in/gp/review/R1UOGC1JPS1M3E,5.0,2019-06-22,https://www.amazon.in/gp/profile/amzn1.account...,Mohan,AGJV5FQHPSNNZ4TTCLA2XQ4TSWIA,Excellent phone with amazing features one plus 6T,Battery life could have been better,Yes,B07DJHY82F,Not Deleted,Midnight Black,8GB/128 GB
2,B07DJHV6VZ,IN,"OnePlus 6T (Mirror Black, 8GB RAM, 128GB Storage)",https://www.amazon.in/gp/review/R252PGXEB4KY1L,5.0,2019-06-22,https://www.amazon.in/gp/profile/amzn1.account...,Suraj,AEEZPQEWKAAKMVWST3SHVVBKOERQ,Awesome phone,Best phone till yet,Yes,B07DJD1Y3Q,Not Deleted,Mirror Black,6GB/128 GB
3,B07DJHV6VZ,IN,"OnePlus 6T (Mirror Black, 8GB RAM, 128GB Storage)",https://www.amazon.in/gp/review/R1EBSSECYNPVCB,1.0,2019-06-22,https://www.amazon.in/gp/profile/amzn1.account...,Sakthi,AFBBATS6CTG6366NWUXKINOW2XQQ,Cheating on the offer,They says ICICI Debit card will have 10% disco...,No,B07DJHV6VZ,Not Deleted,Mirror Black,8GB/128 GB
4,B07DJHV6VZ,IN,"OnePlus 6T (Mirror Black, 8GB RAM, 128GB Storage)",https://www.amazon.in/gp/review/R325E8IVWNOC1L,5.0,2019-06-22,https://www.amazon.in/gp/profile/amzn1.account...,Akash chahar,AGUSFUJAGVSHMNQOZAESMOAOM7AA,True Flagship Killer under 30K,"All in all, this is the best phone you can get...",Yes,B07DJHV6VZ,Not Deleted,Mirror Black,8GB/128 GB


In [0]:
#shape of the data
review_data.shape

(5232, 16)

In [0]:
#taking only reviews from the dataset
review_text = review_data['Review Text']

In [0]:
#lowercasing the text
review_lower = [text.lower() for text in review_text]

In [0]:
#function to remove emojis
def deEmojify(review_list):
    deEmoji = []
    for i in review_list:
        review = i.encode('ascii', 'ignore').decode('ascii')
        deEmoji.append(review)
    return deEmoji

In [0]:
#calling the function to remove emojis
reviews = deEmojify(review_lower)

In [0]:
#cleaning the punctuations in reviews
reviews = [re.sub('[.]+',' ',s) for s in reviews]
reviews = [re.sub('[,]+',' ',s) for s in reviews]
reviews= [re.sub('[\n]+',' ',s) for s in reviews]
reviews= [re.sub("[']+",' ',s) for s in reviews]
reviews = [re.sub('[@,&,!]+', '', s) for s in reviews ]
reviews = [re.sub('[0-9]','',s)for s in reviews]
reviews = [re.sub(r'[.|,|)|(|\|/]',r' ',s)for s in reviews]
reviews = [re.sub(r'[?|!|\'|"|#]',r'',s)for s in reviews]
reviews = [re.sub('[+]+','',s)for s in reviews]
reviews = [re.sub('[:]+','',s)for s in reviews]
reviews = [re.sub('[%]+','',s)for s in reviews]
reviews = [re.sub('[-]+','',s)for s in reviews]

In [0]:
reviews1 = reviews.copy()

In [0]:
#N=nltj packages 
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('brown')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [0]:
#tokenizing the data
tokens=[]
for i in reviews:
    tokenized_word = word_tokenize(i)
    tokens.append(tokenized_word)
#len(tokens)

In [0]:
#tokens

In [0]:
#lementazing the reviews
lem = []
for i in tokens:
  lemmatized_words = [WordNetLemmatizer().lemmatize(x) for x in i]
  lem.append(lemmatized_words)
#lem

In [0]:
#removing the stop words
stop_words = list(stopwords.words("english"))
stop_data = [[word for word in sub if word not in stop_words] for sub in lem]

In [0]:
#de tokenizing the data
de_token=[]
for i in stop_data:
  d = TreebankWordDetokenizer().detokenize(i)
  de_token.append(d)

In [0]:
#removing the numbers in tokens
de_token_reviews = [ re.sub(r'\b\w{1,3}\b', '', s) for s in de_token]


In [0]:
#tokenizing the data
tokens=[]
for i in de_token_reviews:
    tokenized_word = word_tokenize(i)
    tokens.append(tokenized_word)
#len(tokens)

In [0]:
tokens1 = tokens.copy()

In [0]:
#extracting the noun phrases from the text reviews using the text blob
review_blob=[]
for i in tokens:
    blob = TextBlob(str(i))
    noun = blob.noun_phrases
    review_blob.append(noun)
    

In [0]:
#review_blob

In [0]:
#cleaning the nouns
nouns = list(x for l in review_blob for x in l)
nouns=nouns[3:len(nouns)]
nouns = [re.sub("[[  ']+", "" ,s) for s in nouns]
nouns = [re.sub("[]  ']+", "" ,s) for s in nouns]
nouns = np.unique(nouns)
nouns = [re.sub("['*']+","",s) for s in nouns]
nouns = [re.sub("['=']+","",s) for s in nouns]
unique_nouns = [re.sub("[' ']+","",s) for s in nouns]

In [0]:
df_nouns = pd.DataFrame({'noun_col':unique_nouns})

In [0]:
df_nouns=df_nouns.drop([0,1,2],axis=0).head()
df_nouns.head()

,noun_col
3,product
4,aaah
5,abhi
6,absolute
7,accessory


In [0]:
df_nouns.to_csv('nouns.csv')

In [0]:
#creating the sparse matrix
df = pd.DataFrame()
for i in nouns:
  df[i] = np.where(review_data['Review Text'].str.contains(i),1,0)

In [0]:
df.head()

,,product,aaah,abhi,absolute,accessory,achha,adipoli,advance,alert,allow,almost,already,alret,also,always,amazon,amazononeplus,ambient,android,another,anti,apart,apps,around,aswm,atleast,attractive,audio,ausam,average,awasome,awesome,awesometacular,awful,awrsome,awsm,awsme,awsome,awsomw,...,vera,version,veru,verygood,video,vivo,voice,waaooo,wanderfull,want,warp,waste,waterproof,week,weight,weird,well,whatever,whereas,whoever,wife,wish,within,without,wonderful,wonderfull,word,work,world,worse,worst,worth,worthy,would,wowwwww,write,wrost,youtuber,zehrr,zero
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df.to_csv('sparse.csv')

In [0]:
#applying the association rule mining
freq = apriori(df,min_support=0.01,use_colnames=True)
freq.head()

,support,itemsets
0,1.000000,()
1,0.090596,(product)
2,0.034404,(also)
3,0.042622,(awesome)
4,0.034404,(back)


In [0]:
freq.to_csv('frequent_itemsets.csv')

In [0]:
#rules
rules = association_rules(freq,metric='lift',min_threshold=2)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(batter),(also),0.080657,0.034404,0.010512,0.130332,3.788310,0.007737,1.110304
1,(also),(batter),0.034404,0.080657,0.010512,0.305556,3.788310,0.007737,1.323853
2,(battery),(also),0.079511,0.034404,0.010512,0.132212,3.842949,0.007777,1.112709
3,(also),(battery),0.034404,0.079511,0.010512,0.305556,3.842949,0.007777,1.325505
4,(camer),(also),0.108754,0.034404,0.012806,0.117750,3.422613,0.009064,1.094471


In [0]:
rules.to_csv('rules.csv')

In [0]:
#extracting the frequent features using compactness pruning
nlp = spacy.load('en_core_web_sm')
tknzr=TweetTokenizer()
rev=[]
adj=[]
fea=[]
noun=[]
dis=[]
flag=0
two_set = pd.DataFrame(columns=['Review','Feature','Distance'])
features = freq['itemsets']
sd=[]
temp=[]
for x in features:
    temp=[]
    v= str(x)
    rt = v.strip('(')
    rt = rt.strip(')')
    rt = rt.split(',')
    for k in rt:
        temp.append(k.split("'")[1])
    sd.append(temp)
features=sd
#print(two_features)
two_features=[x for x in features if len(x)==2]
#print(two_features)

for i in review_text:
    tokens = tknzr.tokenize(i)
    #print(tokens)
    for feature in two_features:
        #print(feature)
        if feature[0] in tokens and feature[1] in tokens:
            a=tokens.index(feature[0])
            b=tokens.index(feature[1])
            flag = 0
            if abs(a-b)<=4:
                rev.append(i)
                fea.append(feature)
                dis.append(a-b)
                break
            else:
                rev.append(i)
                fea.append(None)
                dis.append(None)
        else:
            flag=-1
    if flag==-1:
        rev.append(i)
        fea.append(None)
        dis.append(None)
two_set['Review']=rev
two_set['Feature']=fea
two_set['Distance']=dis
    

In [0]:
frequent_features={}
a = [str(i) for i in two_set['Feature']]
#print(a)
for i in two_set['Feature']:
  #print(i)
  a.append(str(i))
counter = collections.Counter(a)
for i in counter:
    if counter[i]>2:
        frequent_features[i]=counter[i]
xc=pd.DataFrame.from_dict(counter,orient='index',columns=['Frequency'])
xc.index.names=['FrequentFeatures']
compact_frequent_items=xc.iloc[1:]
compact_frequent_items

,Frequency
FrequentFeatures,
"['phone', 'till']",12
"['battery', 'life']",164
"['camera', 'battery']",100
"['good', 'plus']",14
"['product', 'good']",84
"['finger', 'scanner']",20
"['camera', 'phone']",30
"['performance', 'battery']",22
"['good', 'battery']",88


In [0]:
Creating a dataframe
pd.DataFrame(compact_frequent_items).to_csv('compact_frequent_frequent.csv')

In [0]:
df_nouns = df_nouns.loc[3:]

In [0]:
#reading the reviews, frequent itemsets of apriori and compactness pruning
reviews = pd.read_csv('reviews-all_reviews-default-2019-06-23.csv')
apriori = pd.read_csv('frequent_itemsets.csv')
compact_prune = pd.read_csv('compact_frequent_frequent.csv')

In [0]:
rev_text = reviews['Review Text']
compact_prune.columns=['comp_feat','freq']
apr_text = apriori.itemsets
sd=[]
temp=[]
for i in apr_text:
  temp=[]
  rt = i.strip('[]')
  rt=rt.split(',')
  for k in rt:
    temp.append(k.split("'")[1])
  sd.append(temp)
features = sd

In [0]:
comp_text = compact_prune.comp_feat

In [0]:
#Applying redundancy pruning with features greater than 1
one_features = [x for x in features if len(x)==1]
sd=[]
temp=[]
for x in compact_prune.comp_feat:
  temp=[]
  rt = x.strip('[]')
  rt = rt.split(',')
  for k in rt:
    temp.append(k.split("'")[1])
  sd.append(temp)
two_feature = sd
#len(two_feature)

In [0]:
#creating the data frame
one = pd.DataFrame(one_features)
one.to_csv('one_features.csv')


In [0]:
#calculating p value
p_value=dict()
for i in reviews.iterrows():
  sent_token = list(map(str.lower,nltk.sent_tokenize(i[1][10])))
  #print(sent_token)
  for j in sent_token:
    wrd_tok = nltk.wordpunct_tokenize(j)
    #print(wrd_tok)
    tokens = [i.strip("!@#$%&*?.,/-;:\'") for i in wrd_tok]
    #print(tokens)
    for one in one_features:
      if one[0] in tokens:
        p_value[one[0]] = p_value.setdefault(one[0], 0) + 1  
      #print(p_value[one[0]])

In [0]:
#sorting the p values
sorted_pvalue = sorted(p_value.items(), reverse=True, key=itemgetter(1))

In [0]:
#checking p value with features 
updated_pval = dict()
for index1,one in enumerate(sorted_pvalue):
  p_val = sorted_pvalue[index1][1]
  for index2,two in enumerate(two_feature):
    if one[0] in two:
      p_val = p_val - compact_prune.freq[index2]
  updated_pval[one[0]] = p_val

In [0]:
#sorting the updated p values
updated_pval = sorted(updated_pval.items(), reverse=True, key=itemgetter(1))

In [0]:
#creating the data frames
df_pvalue = pd.DataFrame.from_dict(updated_pval)
df_pvalue.columns = ['Features','p_value']
df_pvalue.to_csv('compact_prune_words.csv')


In [0]:
df_pvalue.head()

,Features,p_value
0,,6416
1,phone,920
2,good,736
3,oneplus,452
4,product,416


In [0]:
compact_prune = xc.iloc[1:].copy()
compact_prune.head()

,Frequency
FrequentFeatures,
"['phone', 'till']",12
"['battery', 'life']",164
"['camera', 'battery']",100
"['good', 'plus']",14
"['product', 'good']",84


In [0]:
df_pvalue['Features'] = df_pvalue['Features']

In [0]:
final_features = pd.concat([compact_prune,df_pvalue])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [0]:
final_features.to_csv('final_features.csv')

In [0]:
#manual labels for the categories
possible_labels= {
'Display':['time screen','screen','phone screen','screen quality','display quality','performance display','screen','phone screen','performance display','life display','phone display','display price','display'],
'Audio':['sound quality','volume','speaker sound','speaker quality','time sound','Jack','sound','head phone','phone sound','speaker sound','speaker quality','speaker'],
'Body':['build','build quality','phone look','prodcut quality','phone quality','mobile quality',"device quality"],
'Memory':['card slot','ram','memory'],
'Platform':['app camera','camera software','processor quality','processor','software','phone software','phone update','phone processor','android phone','android'],
'Camera':['video quality','camera mobile','camera experience','camera price','product camera','camera pro','beast camera','camera feature','charge camera','camera','camera quality','video camera','camera mode','face camera','phone camera','camera issue','light camera','speed camera',"picture quality","camera performance"],
'Back Camera':['back camera','pixel','rear camera'],
'Front Camera':['front camera','face camera'],
'Misc':['color'],
'Issues':['issue','phone problem','phone issue','month issue','camera issue','day issue','life issue','issue quality','battery issue'],

'Features':['fingerprint','face unlock','phone unlock','unlock'],
'Battery':['charging phone','battery feature','battery usage','battery','charging','battery problem','charger','charge','battery usage','charge battery','battery charging','dash charger','dash charging','battery back','battery life','battery backup','battery product','life battery','app battery','battery back','battery mobile','battery bit','battery speed','battery performance','battery phone','battery charger','battery quality'],
'Price':['display price','month price','budget','price','flagship price','phone value','camera value','product price','range price','mobile price','phone price','range phone'],
'Performance':['sound performance','performance device','speed','phone performance','speed performance','performance mobile']
}

In [0]:
#cleaning the reviews
reviews1 = [re.sub("[^a-zA-Z]", " ",s) for s in reviews1]

In [0]:
#extracting opinions from the reviews with possible albels
def opinion_sent(review):
    opinion=[]
    sent=sent_tokenize(review.lower())
    for i in sent:
        opinion_sent=[]
        cat=[]
        for label in possible_labels.keys():
            if (any(re.search(feature,i) for feature in possible_labels[label])):
                opinion_sent.append(i)   
        opinion.append(list(set(opinion_sent))) 
        flatted_opinion =[y for x in opinion for y in x]
    return(', '.join(flatted_opinion))

In [0]:
opinion_df = pd.DataFrame()

In [0]:
opinion_df['Review Text'] = review_data['Review Text']

In [0]:
#applying the opinion sent function
opinion_df["opinion_sent"] = opinion_df['Review Text'].apply(opinion_sent)

In [0]:
#Counting the number of categories
def categories(review):
  category=[]
  sent = sent_tokenize(review.lower())
  for i in sent:
    opinion_sent=[]
    cat=[]
    for label in possible_labels.keys():
      if (any(re.search(feature,i) for feature in possible_labels[label])):
        cat.append(label)
    category.append(cat)
    flatted_cat =[y for x in category for y in x]
    unique=list(set(flatted_cat))
  return(', '.join(unique))

In [0]:
#applying the categories function
opinion_df['categories'] = opinion_df['Review Text'].apply(categories)

In [0]:
opinion_df = opinion_df[opinion_df.categories!=""]

In [0]:
opinion_df.head()

,Review Text,opinion_sent,categories
1,Battery life could have been better,battery life could have been better,Battery
4,"All in all, this is the best phone you can get...",pros:\n1) great notch\n2) face unlock is real ...,"Display, Battery, Camera, Platform, Memory, Fe..."
5,Ideal for budget phone,ideal for budget phone,Price
7,i like everything. natural like camera quality...,"natural like camera quality, decent battery li...","Battery, Camera"
8,Camera poor,camera poor,Camera


In [0]:
#counting the number of categories
def count(opinion):
  features=[]
  count=[]
  for label in possible_labels.keys():
    features.append(label)
    count.append(opinion.loc[:,"categories"].apply(lambda x: x.count(str(label))).sum())
  value = pd.DataFrame()
  value['Labels'] = features
  value['Count'] = count
  return(value)

In [0]:
#def no_adjective(opinion_sent):
#    adj_count=0
#    #for i in opinion_sent:
#    parse=nlp(opinion_sent)
#    for j in parse:
#        if (re.get_pos(j)=="ADJ"):
#            adj_count+=1
#    return(adj_count)

In [0]:
#the number of adjectives from opinion sent
def no_adject(opinion_sent):
  adj_count=0
  adjectives = deEmojify(opinion_sent)
  adjectives = [re.sub('[.]+',' ',s) for s in adjectives]
  adjectives = [re.sub('[,]+',' ',s) for s in adjectives]
  adjectives= [re.sub('[\n]+',' ',s) for s in adjectives]
  adjectives= [re.sub("[']+",' ',s) for s in adjectives]
  adjectives = [re.sub('[@,&,!]+', '', s) for s in adjectives ]
  adjectives = [re.sub('[0-9]','',s)for s in adjectives]
  adjectives = [re.sub(r'[.|,|)|(|\|/]',r' ',s)for s in adjectives]
  adjectives = [re.sub(r'[?|!|\'|"|#]',r'',s)for s in adjectives]
  adjectives = [re.sub('[+]+','',s)for s in adjectives]
  adjectives = [re.sub('[:]+','',s)for s in adjectives]
  adjectives = [re.sub('[%]+','',s)for s in adjectives]
  adjectives = [re.sub('[-]+','',s)for s in adjectives]
  adjective_tokens=[]
  for i in adjectives:
    tokenized_word = word_tokenize(i)
    adjective_tokens.append(tokenized_word)
  #import nltk
  #nltk.download('averaged_perceptron_tagger')
  pos_adj=[]
  for i in adjective_tokens:
    pos = nltk.pos_tag(i)
    pos_adj.append(pos)
  adject=[]
  for i in pos_adj:
    temp = [j[0] for j in i if j[1].startswith('JJ')]
    adj_count+=1
 
  return(adj_count)
    
  

In [0]:
#applying the no_adject function
opinion_df['No_adj'] = opinion_df['opinion_sent'].apply(no_adject)

In [0]:
opinion_df.head()

,Review Text,opinion_sent,categories,No_adj
1,Battery life could have been better,battery life could have been better,Battery,35
4,"All in all, this is the best phone you can get...",pros:\n1) great notch\n2) face unlock is real ...,"Display, Battery, Camera, Platform, Memory, Fe...",361
5,Ideal for budget phone,ideal for budget phone,Price,22
7,i like everything. natural like camera quality...,"natural like camera quality, decent battery li...","Battery, Camera",52
8,Camera poor,camera poor,Camera,11


In [0]:
#calculating the raank
def ranking(review):
  labels = review['categories']
  cat = labels.split(",")
  rank = len(cat)*review['No_adj']
  return(rank)

In [0]:
opinion_df['Ranking'] = opinion_df.apply(ranking,axis=1)

In [0]:
opinion_df.head(5)

,Review Text,opinion_sent,categories,No_adj,Ranking
1,Battery life could have been better,battery life could have been better,Battery,35,35
4,"All in all, this is the best phone you can get...",pros:\n1) great notch\n2) face unlock is real ...,"Display, Battery, Camera, Platform, Memory, Fe...",361,2166
5,Ideal for budget phone,ideal for budget phone,Price,22,22
7,i like everything. natural like camera quality...,"natural like camera quality, decent battery li...","Battery, Camera",52,104
8,Camera poor,camera poor,Camera,11,11


In [0]:
opinion_df.to_csv('opinion_table.csv')